In [5]:
# Web Scraping -> euroagd.com.pl

import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import re
import pandas as pd

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE


url = "https://www.euro.com.pl/laptopy-i-netbooki.bhtml"
table_euro = pd.DataFrame()
while True:
    html = urllib.request.urlopen(url, context=ctx).read()
    soup = BeautifulSoup(html, "html.parser")
    products = soup.findAll("div", {"class":"product-for-list"})
    
    name_values = []
    parameters_names = []
    parameters_values = []
    for i in products:
        name = i.img["alt"]
        name_values.append(name)
        attributes = i.findAll("div", {"class":"product-attributes"})[0].findAll("div", {"class":"attributes-row"})
        for parameter in attributes:
            namen = parameter.find("span", {"class":"attribute-name"}).text.strip()
            parameters_names.append(namen)
            value = parameter.find("span", {"class": "attribute-value"}).text.strip()
            parameters_values.append(value)
    
    
    screen_values = []
    processor_values = []
    memory_values = []
    graphics_values = []
    disc_values = []
    system_values = []
    for i in range(len(parameters_names)):
        if parameters_names[i] == "Ekran":
            parameter = parameters_values[i]
            screen_values.append(parameter)
        if parameters_names[i] == "Procesor":
            parameter = parameters_values[i]
            processor_values.append(parameter)
        if parameters_names[i] == "Pamięć":
            parameter = parameters_values[i]
            memory_values.append(parameter)
        if parameters_names[i] == "Grafika":
            parameter = parameters_values[i]
            graphics_values.append(parameter)
        if parameters_names[i] == "Dysk":
            parameter = parameters_values[i]
            disc_values.append(parameter)
        if parameters_names[i] == "System operacyjny":
            parameter = parameters_values[i]
            system_values.append(parameter)
    
    length = len(system_values)
        
    name_values = []
    price_values = []
    brand_values = []
    id_values = []
    tags =soup("script")
    for tag in tags:
        type_var = tag.get("type", None)
        if type_var is None:
            continue
        if "".join(type_var) == "text/javascript":
            string = str(tag)
            try:
                lista = re.findall("list: \"(.*)\"", string)[0]
                id_i = re.findall("id: \"(.*)\"", string)[0]
                if id_i not in id_values:
                    id_values.append(id_i)
                    name = re.findall("name: \"(.*)\"", string)[0]
                    price = re.findall("price: \"(.*)\"", string)[0]
                    brand = re.findall("brand: \"(.*)\"", string)[0]
                    name_values.append(name)
                    price_values.append(price)
                    brand_values.append(brand)
            except IndexError:
                continue
        
    data_var = {"Name": name_values[:length], "Brand": brand_values[:length], "Price": price_values[:length], "Screen": screen_values, "Processor": processor_values, "Memory": memory_values, "Graphics": graphics_values, "Disc": disc_values, "System": system_values}
    euro = pd.DataFrame(data=data_var)
    
    table_euro = pd.concat([table_euro, euro], ignore_index=True)

    
    try:
        url_end = soup("a", {"class": "paging-next selenium-WC-paging-next-button"})[0]["href"]
        url = "https://www.euro.com.pl" + url_end
    except:
        break
        

In [4]:
# Web Scraping -> https://www.komputronik.pl

import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import re
import pandas as pd

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url = "https://www.komputronik.pl/category/5022/laptopy.html"
table_komp = pd.DataFrame()
y = 1

while True:
    print(y)
    user_agent = 'obejscie'
    headers={'User-Agent':user_agent,} 

    request = urllib.request.Request(url,None,headers)
    response = urllib.request.urlopen(request)
    data = response.read() 

    soup = BeautifulSoup(data, "html.parser")

    a = soup.findAll("div", {"class": "pe2-desc"})

    if len(a) != 20 and url != "https://www.komputronik.pl/category/5022/laptopy.html?showBuyActiveOnly=0&p=92":
        p = soup("div", {"class": "inline-features"})[:20]

        parameters = []
        for i in p:
            parameters.append(i.span.text.strip())

        graphics_values = []
        resolution_values = []
        processor_values = []
        ram_values = []
        diskssd_values = []
        system_values = []

        for i in parameters:
            list_var = i.replace(" ", "").split("|")
            graphics_values.append(list_var[0])
            resolution_values.append(list_var[1])
            if len(list_var) != 6:
                processor_values.append(None)
                ram_values.append(list_var[2])
                diskssd_values.append(list_var[3])
                system_values.append(list_var[4])
            else:
                processor_values.append(list_var[2])
                ram_values.append(list_var[3])
                diskssd_values.append(list_var[4])
                system_values.append(list_var[5])

    else:


        parameters_names = []
        parameters_values = []
        set_var = ["System operacyjny", "Rozdzielczość", "Typ procesora", "Wielkość pamięci RAM", "Pojemność dysku SSD", "Pojemność dysku HDD", "Karta graficzna"]
        set_var = set(set_var)
        p = 0

        for laptop in a:
            p += 1
            if p == 21:
                break
            changes = []
            b = laptop.findAll("li")
            for attribute in b:

                value = attribute.text.strip()
                if re.search("^System operacyjny:", value):
                    value = re.findall("System operacyjny: (.*)", " ".join(value.split()))[0]
                    parameters_values.append(value)
                    parameters_names.append("System operacyjny")
                    changes.append("System operacyjny")
                if re.search("^Rozdzielczość:", value):
                    value = re.findall("Rozdzielczość: (.*)", " ".join(value.split()))[0]
                    parameters_values.append(value)
                    parameters_names.append("Rozdzielczość")
                    changes.append("Rozdzielczość")
                if re.search("^Typ procesora:", value):
                    value = re.findall("Typ procesora: (.*)", " ".join(value.split()))[0]
                    parameters_values.append(value)
                    parameters_names.append("Typ procesora")
                    changes.append("Typ procesora")
                if re.search("^Wielkość pamięci RAM:", value):
                    value = re.findall("Wielkość pamięci RAM: (.*)", " ".join(value.split()))[0]
                    parameters_values.append(value)
                    parameters_names.append("Wielkość pamięci RAM")
                    changes.append("Wielkość pamięci RAM")
                if re.search("^Pojemność dysku SSD:", value):
                    value = re.findall("Pojemność dysku SSD: (.* GB)", " ".join(value.split()))[0]
                    parameters_values.append(value)
                    parameters_names.append("Pojemność dysku SSD")
                    changes.append("Pojemność dysku SSD")
                if re.search("^Pojemność dysku HDD:", value):
                    value = re.findall("Pojemność dysku HDD: (.* GB)", " ".join(value.split()))[0]
                    parameters_values.append(value)
                    parameters_names.append("Pojemność dysku HDD")
                    changes.append("Pojemność dysku HDD")
                if re.search("^Karta graficzna:", value):
                    value = re.findall("Karta graficzna: (.*)", " ".join(value.split()))[0]
                    parameters_values.append(value)
                    parameters_names.append("Karta graficzna")
                    changes.append("Karta graficzna")


            changes1 = set(changes)
            difference = set_var.difference(changes1)
            if len(difference) !=  0:
                for i in difference:
                    parameters_names.append(i)
                    parameters_values.append(None)




        system_values = []
        resolution_values = []
        processor_values = []
        ram_values = []
        diskssd_values = []
        dischdd_values = []
        graphics_values = []

        for i in range(len(parameters_names)):
            if parameters_names[i] == "System operacyjny":
                parameter = parameters_values[i]
                system_values.append(parameter)
            if parameters_names[i] == "Rozdzielczość":
                parameter = parameters_values[i]
                resolution_values.append(parameter)
            if parameters_names[i] == "Typ procesora":
                parameter = parameters_values[i]
                processor_values.append(parameter)
            if parameters_names[i] == "Wielkość pamięci RAM":
                parameter = parameters_values[i]
                ram_values.append(parameter)
            if parameters_names[i] == "Pojemność dysku SSD":
                parameter = parameters_values[i]
                diskssd_values.append(parameter)
            if parameters_names[i] == "Pojemność dysku HDD":
                parameter = parameters_values[i]
                dischdd_values.append(parameter)
            if parameters_names[i] == "Karta graficzna":
                parameter = parameters_values[i]
                graphics_values.append(parameter)




    laptop_names = []
    if url == "https://www.komputronik.pl/category/5022/laptopy.html":
        for i in range(20):
            string = "blank-link at-product-name-" + str(i)
            name = soup.findAll("a", {"class": string})[0].text.strip()
            laptop_names.append(name)
    else:
        laptop_names = []
        names = soup.findAll("li", {"class": "product-entry2"})
        for name in names:
            name1 = name.a.text.strip()
            laptop_names.append(name1)
        laptop_names = laptop_names[:-1]


    laptop_prices = []
    for i in range(20):
        string = "proper at-gross-price-" + str(i)
        price = soup.findAll("span", {"class": string})[0].text.strip()
        price = price.replace("\xa0", "").replace("zł", "")
        laptop_prices.append(price)


    laptop_models = []
    for model in laptop_names:
        position = model.find(" ")
        modello = model[:position]
        laptop_models.append(modello)



    data_var = {"Name": laptop_names, "Brand": laptop_models, "Price": laptop_prices, "Screen": resolution_values, "Processor": processor_values, "Memory": ram_values, "Graphics": graphics_values, "Disc": diskssd_values, "System": system_values}
    komp = pd.DataFrame(data=data_var)

    table_komp = pd.concat([table_komp, komp], ignore_index=True)


    y += 1  
    url = "https://www.komputronik.pl/category/5022/laptopy.html?showBuyActiveOnly=0&p=" + str(y)
    if y == 93:
        break



In [ ]:
# Modifying table_komp table

table_komp_v1 = table_komp.loc[(table_komp.Graphics.str.contains("Corei")) | (table_komp.Graphics.str.contains("AMD")) | (table_komp.Graphics.str.contains("Pentium")) | (table_komp.Graphics.str.contains("Ryzen")) | (table_komp.Graphics.str.contains("Celeron")) | (table_komp.Graphics.str.contains("MicrosoftSQ1")), :]
table_komp_v2 = table_komp.loc[(table_komp.Graphics.str.contains("Corei") == False) & (table_komp.Graphics.str.contains("AMD") == False) & (table_komp.Graphics.str.contains("Pentium") == False) & (table_komp.Graphics.str.contains("Ryzen") == False) & (table_komp.Graphics.str.contains("Celeron") == False) & (table_komp.Graphics.str.contains("MicrosoftSQ1") == False), :]

table_komp_v1.rename(columns = {"Processor": "Graphics", "Graphics": "Processor"}, inplace=True)
table_komp_v1 = table_komp_v1.iloc[:, [0, 1, 2, 3, 6, 5, 4, 7, 8]]

table_komp_final = pd.concat([table_komp_v1, table_komp_v2], ignore_index=True)

In [ ]:
# Identifying laptops appearing on both websites

table_euro.merge(table_komp_final, left_on="Name", right_on="Name", suffixes=('_euro', '_komputronik'))

In [ ]:
# Saving laptop prices

table_euro_final = table_euro.iloc[: ,[0, 2]]
table_komp_final1 = table_komp_final.iloc[: ,[0, 2]]
table_euro_final.to_csv("euro.csv")
table_komp_final1.to_csv("komputronik.csv")